In [90]:
#21st Dec'24
import pandas as pd
data = pd.read_csv("/content/SMSSpamCollection.txt", sep="\t",names=['label','message'])
data.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [91]:
data.isna().sum()

,0
label,0
message,0


In [92]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [93]:
corpus = []
for i in range(0,len(data)):
  review = re.sub('[^a-zA-Z]',' ',data['message'][i]) #removing special characters
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words("english")]
  review = ' '.join(review)
  corpus.append(review)


In [94]:
len(corpus)

5572

In [95]:
#SPAM HAM CLASSIFICATION USING BoW

In [96]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500,ngram_range=(1,2))
X = cv.fit_transform(corpus).toarray()

In [114]:
import numpy as np
np.set_printoptions(edgeitems=30,linewidth=100000,formatter=dict(float=lambda x: "%.3g" % x))
X.shape

(5572, 2500)

In [98]:
cv.vocabulary_

{'go': 795,
 'point': 1607,
 'crazi': 456,
 'avail': 114,
 'bugi': 225,
 'great': 852,
 'world': 2429,
 'la': 1103,
 'cine': 352,
 'got': 840,
 'wat': 2337,
 'ok': 1481,
 'lar': 1114,
 'joke': 1071,
 'wif': 2395,
 'free': 709,
 'entri': 612,
 'wkli': 2419,
 'comp': 406,
 'win': 2400,
 'cup': 471,
 'final': 677,
 'st': 1979,
 'may': 1269,
 'text': 2083,
 'receiv': 1704,
 'question': 1665,
 'std': 1992,
 'txt': 2196,
 'rate': 1682,
 'appli': 78,
 'free entri': 714,
 'entri wkli': 614,
 'std txt': 1993,
 'txt rate': 2202,
 'rate appli': 1684,
 'dun': 573,
 'say': 1803,
 'earli': 579,
 'alreadi': 50,
 'nah': 1392,
 'think': 2103,
 'goe': 815,
 'usf': 2270,
 'live': 1178,
 'around': 90,
 'though': 2111,
 'freemsg': 721,
 'hey': 927,
 'darl': 487,
 'week': 2363,
 'word': 2425,
 'back': 131,
 'like': 1164,
 'fun': 743,
 'still': 1995,
 'tb': 2063,
 'xxx': 2465,
 'send': 1838,
 'even': 622,
 'brother': 214,
 'speak': 1963,
 'treat': 2170,
 'per': 1550,
 'request': 1741,
 'mell': 1281,
 'oru': 

In [99]:
data['label'].value_counts()

,count
label,
ham,4825
spam,747


In [100]:
#ham:0, spam:1
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y = le.fit_transform(data['label'])
Y = pd.DataFrame(data=Y,columns=['label'])
Y.value_counts()

,count
label,
0,4825
1,747


In [101]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)


In [102]:
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(4457, 2500) (1115, 2500) (4457, 1) (1115, 1)


In [103]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train,Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [104]:
Y_pred = spam_detect_model.predict(X_test)

In [105]:
from sklearn.metrics import accuracy_score, classification_report


In [106]:
accuracy_score(Y_test,Y_pred)

0.9874439461883409

In [107]:
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       955
           1       0.98      0.93      0.96       160

    accuracy                           0.99      1115
   macro avg       0.98      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115



In [108]:
#SPAM HAM CLASSIFICATION USING TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
tv = CountVectorizer(max_features=2500,ngram_range=(1,2))
X = tv.fit_transform(corpus).toarray()

In [109]:
#tv.vocabulary_
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [110]:
from sklearn.naive_bayes import MultinomialNB
spam_tfidf_model = MultinomialNB().fit(X_train,Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [111]:
Y_pred = spam_tfidf_model.predict(X_test)

In [112]:
accuracy_score(Y_test,Y_pred)

0.9874439461883409

In [113]:
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       955
           1       0.98      0.93      0.96       160

    accuracy                           0.99      1115
   macro avg       0.98      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115

